## Building a web scrapper that extracts product search results from amazon.com <br> 

In this notebook, I am building a web scrapper for amazon.com. Given a search term, this scrapper will extract records from all pages in a search result and store them in a csv file. <br>As an example, we will be using "bike lock" as search term.

### Installing and importing the packages/libraries

In [1]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install bs4

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

In [4]:
# Creating an instance of the webdriver to start it up
driver = webdriver.Chrome("C:/Users/amari/Downloads/chromedriver_win32/chromedriver.exe")

C:\Users\amari\AppData\Local\Temp\ipykernel_16248\418142941.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/Users/amari/Downloads/chromedriver_win32/chromedriver.exe")


In [5]:
#navigate to the amazon website

url = 'https://www.amazon.com'
driver.get(url)

In [6]:
# writing a function that will generate a url based on the search terms provided

def get_url(search_term):
    """Generate a url from search term"""
    template = 'https://www.amazon.com/s?k={}&crid=2LEXIUFP0SED4&sprefix={}%2Caps%2C148&ref=nb_sb_noss_2'
    search_term = search_term.replace(' ', '+')
    return template.format(search_term, search_term)     #since the search term appears twice in the url when tried on amazon.com

In [7]:
# Creating a url for the bike lock search

url = get_url("bike lock")
print(url)

https://www.amazon.com/s?k=bike+lock&crid=2LEXIUFP0SED4&sprefix=bike+lock%2Caps%2C148&ref=nb_sb_noss_2


In [8]:
driver.get(url)

### Extracting the content of the page from the html in the background

In [9]:
#creating soup object which will parse the html content from the page source
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [10]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})
results

[<div class="sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 AdHolder sg-col s-widget-spacing-small sg-col-4-of-20" data-asin="B07T3F6JST" data-component-id="19" data-component-type="s-search-result" data-index="1" data-uuid="24fbccce-5092-4a1c-9374-0957627813ba"><div class="sg-col-inner"><div cel_widget_id="MAIN-SEARCH_RESULTS-1" class="s-widget-container s-spacing-small s-widget-container-height-small celwidget slot=MAIN template=SEARCH_RESULTS widgetId=search-results_1" data-csa-c-id="4svha9-4irizu-62gd2h-vfwwt4" data-csa-c-item-id="amzn1.asin.1.B07T3F6JST" data-csa-c-pos="1" data-csa-c-type="item" data-csa-op-log-render="">
 <div class="rush-component s-expand-height" data-component-id="20" data-component-props='{"percentageShownToFire":"50","batchable":true,"requiredElementSelector":".s-image:visible","url":"https://unagi-na.amazon.com/1/events/com.amazon.eel.SponsoredProductsEventTracking.prod?qualifier=1663423799&amp;id=2309119300619425&amp;widgetName=sp_atf&amp;adId=20002282

In [11]:
len(results)

60

### Prototyping the extraction of a single record

In [12]:
item = results[0]

In [13]:
atag = item.h2.a

In [14]:
description = atag.text.strip() 
description

'Amazon Basics 6 ft. Adjustable Keyed Bike Cable Lock, Black, 1-Pack'

In [15]:
url = 'https://www.amazon.com' + atag.get('href')
url

'https://www.amazon.com/sspa/click?ie=UTF8&spc=MToyMzA5MTE5MzAwNjE5NDI1OjE2NjM0MjM3OTk6c3BfYXRmOjIwMDAyMjgyMzYzNzIzMTo6MDo6&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&url=%2FAmazonBasics-Adjustable-Keyed-Cable-1-Pack%2Fdp%2FB07T3F6JST%2Fref%3Dsr_1_1_sspa%3Fcrid%3D2LEXIUFP0SED4%26keywords%3Dbike%2Block%26qid%3D1663423799%26sprefix%3Dbike%2Block%252Caps%252C148%26sr%3D8-1-spons%26psc%3D1'

In [16]:
# getting the price
price_parent = item.find('span', 'a-price')
price = price_parent.find('span', 'a-offscreen').text

In [17]:
# getting the ratings. Inspecting the stars shows that ratings are in the i tag

rating = item.i.text

In [18]:
#getting the reviews

item.find('span', {'class':'a-size-base s-underline-text'}).text

'1,303'

### Generalizing the patttern of extraction within a function which can be applied to all the records on a page

Here I incorporated some error handling. 5 attributes will be extracted from each record, but since some attributes may be missing in some records, AttributeError will need to be accomodated.

Add error handling: The above code assumes each record contains all 5 attributes, but that's not the case

In [19]:
def extract_record(item):
    """Extract and return data from a single record"""
   
    # description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        #price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        #rating & review(rank)
        rating = item.i.text
        review_count = item.find('span', {'class':'a-size-base s-underline-text'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, price, rating, review_count, url)
    
    return result

In [20]:
# apply the pattern to all records on the page 

records =[]
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:   #ie, if record has something in it
        records.append(record)

In [21]:
# viewing the first record
records[0]

('Amazon Basics 6 ft. Adjustable Keyed Bike Cable Lock, Black, 1-Pack',
 '$13.91',
 '4.6 out of 5 stars',
 '1,303',
 'https://www.amazon.com/sspa/click?ie=UTF8&spc=MToyMzA5MTE5MzAwNjE5NDI1OjE2NjM0MjM3OTk6c3BfYXRmOjIwMDAyMjgyMzYzNzIzMTo6MDo6&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&url=%2FAmazonBasics-Adjustable-Keyed-Cable-1-Pack%2Fdp%2FB07T3F6JST%2Fref%3Dsr_1_1_sspa%3Fcrid%3D2LEXIUFP0SED4%26keywords%3Dbike%2Block%26qid%3D1663423799%26sprefix%3Dbike%2Block%252Caps%252C148%26sr%3D8-1-spons%26psc%3D1')

### Getting the next page

On the amazon page, if you click on 'Next' and oberve the url of the next page, you notice a parameter for page number. We can add this page parameter to the url using string formatting. Amazon search gives a maximum of 20 pages

In [22]:
def get_url(search_term):
    """Generate a url from search term"""
    template = 'https://www.amazon.com/s?k={}&crid=2LEXIUFP0SED4&sprefix={}%2Caps%2C148&ref=nb_sb_noss_2'
    search_term = search_term.replace(' ', '+')
    
    # add term query to url
    url = template.format(search_term, search_term)
    
    #add page query placeholder
    url += '&page{}'  #this gives a place to insert the next page number (with string formatting)
    
    return url

### Putting all together

In [23]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver


def get_url(search_term):
    """Generate a url from search term"""
    template = 'https://www.amazon.com/s?k={}&crid=2LEXIUFP0SED4&sprefix={}%2Caps%2C148&ref=nb_sb_noss_2'
    search_term = search_term.replace(' ', '+')
    
    # add term query to url
    url = template.format(search_term, search_term)
    
    #add page query placeholder
    url += '&page{}'  #this gives a place to insert the next page number (with string formatting)
    
    return url

def extract_record(item):
    """Extract and return data from a single record"""
   
    # description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        #price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        #rating & review(rank)
        rating = item.i.text
        review_count = item.find('span', {'class':'a-size-base s-underline-text'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, price, rating, review_count, url)
    
    return result

def main(search_term):
    """Run main program routine"""
    #startup the web driver
    driver = webdriver.Chrome("C:/Users/amari/Downloads/chromedriver_win32/chromedriver.exe")
    
    records = []
    url = get_url(search_term)
    
    for page in range (1, 21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
    driver.close()
    
    #save data to csv
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'url'])
        writer.writerows(records)
    

In [24]:
main('bike lock')

C:\Users\amari\AppData\Local\Temp\ipykernel_16248\3805914634.py:49: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/Users/amari/Downloads/chromedriver_win32/chromedriver.exe")
